In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26680305
paper_name = 'wu_hopper_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [21]:
original_data = pd.read_excel('raw_data/Table_S1.xlsx', sheet_name='Table 1', skiprows=1)

In [22]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 105 x 12


In [23]:
original_data.head()

,Gene Name,Systemic Name,Unnamed: 2,Phenotypes1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,"Northern data location\n(Figure #, gel#)",Unnamed: 9,Unnamed: 10,Confirmation
0,AMD1,YML035C,NaN,Type 1/2,NaN,NaN,NaN,NaN,"S1-3, JW 196",NaN,NaN,MATa vs. MATα
1,AMD2,YDR242W,NaN,Type 1/2,NaN,NaN,NaN,NaN,"S1-3, JW 198",NaN,NaN,MATa vs. MATα
2,APL5,YPL195W,NaN,Type 1/2,NaN,NaN,NaN,NaN,"S1-2, JW 190",NaN,NaN,MATa vs. MATα
3,ARA1,YBR149W,NaN,Type 1/2,NaN,NaN,NaN,NaN,"S1-3, JWl 196",NaN,NaN,MATa vs. MATα
4,ARL1,YBR164C,NaN,Type 1/2,NaN,NaN,NaN,NaN,"S1-3, JW 196",NaN,NaN,MATa vs. MATα


In [24]:
original_data['orf'] = original_data['Systemic Name'].astype(str)

In [25]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [26]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [27]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Gene Name, Systemic Name, Unnamed: 2, Phenotypes1, Unnamed: 4, Unnamed: 5, Unnamed: 6, Unnamed: 7, Northern data location
(Figure #, gel#), Unnamed: 9, Unnamed: 10, Confirmation, orf]
Index: []


In [30]:
ph = np.unique(original_data['Phenotypes1'].values)

In [31]:
for d in ph:
    original_data[d] = 0
    original_data.loc[original_data['Phenotypes1']==d,d] = 1

In [33]:
original_data.set_index('orf', inplace=True)

In [34]:
original_data = original_data[ph].copy()

In [35]:
original_data = original_data.groupby(original_data.index).mean()

In [36]:
original_data.shape

(105, 6)

# Load & process tested strains

In [38]:
tested = pd.read_excel('raw_data/YSC1053.YKO.mat_a.v1.0.xlsx', sheet_name='mat_a_obs')

In [39]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,notes
0,338.0,YAL068C,BY4741,01_1,1.0,A,2.0,NaN
1,339.0,YAL067C,BY4741,01_1,1.0,A,3.0,NaN
2,340.0,YAL066W,BY4741,01_1,1.0,A,4.0,NaN
3,341.0,YAL065C,BY4741,01_1,1.0,A,5.0,NaN
4,345.0,YAL062W,BY4741,01_1,1.0,A,6.0,NaN


In [40]:
tested['orf'] = tested['ORF name'].astype(str)

In [41]:
tested['orf'] = clean_orf(tested['orf'])

In [44]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [45]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [46]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name strain batch  plate  row  column notes  orf
index_input                                                                 
4705                NaN      NaN    NaN   NaN    NaN  NaN     NaN   NaN  NAN


In [47]:
tested = tested.loc[t,:]

In [48]:
tested_orfs = np.unique(tested['orf'].values)

In [49]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [52]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [54]:
original_data.head()

,Type 1/2,Type 1/2 and an extra band below I,Type 2*,Type 4/5/6,Type 7,Type 9
orf,,,,,,
YAL002W,0,0,0,0,0,0
YAL004W,0,0,0,0,0,0
YAL005C,0,0,0,0,0,0
YAL007C,0,0,0,0,0,0
YAL008W,0,0,0,0,0,0


# Prepare the final dataset

In [55]:
data = original_data.copy()

In [56]:
dataset_ids = [4952,11779,11780,11781,11782,11783]
datasets = datasets.reindex(index=dataset_ids)

In [57]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [58]:
data.head()

dataset_id,4952,11779,11780,11781,11782,11783
data_type,value,value,value,value,value,value
orf,,,,,,
YAL002W,0,0,0,0,0,0
YAL004W,0,0,0,0,0,0
YAL005C,0,0,0,0,0,0
YAL007C,0,0,0,0,0,0
YAL008W,0,0,0,0,0,0


## Subset to the genes currently in SGD

In [59]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [60]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4952,11779,11780,11781,11782,11783
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
2,YAL002W,0,0,0,0,0,0
1863,YAL004W,0,0,0,0,0,0
4,YAL005C,0,0,0,0,0,0
5,YAL007C,0,0,0,0,0,0
6,YAL008W,0,0,0,0,0,0


# Normalize

In [61]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [62]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [63]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4952,11779,11780,11781,11782,11783,4952,11779,11780,11781,11782,11783
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
2,YAL002W,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1863,YAL004W,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,YAL005C,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,YAL007C,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
6,YAL008W,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


# Print out

In [64]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [65]:
from IO.save_data_to_db3 import *

In [66]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 26680305...
Inserting the new data...


100%|██████████| 6/6 [00:43<00:00,  7.30s/it]

Updating the data_modified_on field...
